In [1]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.core.rust.model import TriggerType, TimeInForce
from nautilus_trader.model import TradeTick
from nautilus_trader.model import InstrumentId

In [2]:
catalog = ParquetDataCatalog("/home/p/p-dev/nautilus/catalog")
instrument_id = InstrumentId.from_str("APTUSDT-PERP.BINANCE")

In [3]:
venue = BacktestVenueConfig(
    name="BINANCE",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USDT",
    starting_balances=["100 USDT"],
    default_leverage=10
)

In [4]:
data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=TradeTick,
    instrument_ids=[instrument_id],
    start_time="2024-12-10",
    end_time="2025-01-10"
)

In [5]:
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
              "instrument_id": instrument_id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="DEBUG"),
)

In [6]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.core.rust.common import LogColor
from nautilus_trader.model.events import PositionOpened, PositionChanged, OrderAccepted
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig



class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 500
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        self.log.info("Initialization of MACDStrategy")
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        # Convenience
        self.position: Position | None = None

    
    def on_start(self):
        self.log.info("Start of MACDStrategy")
        
        self.instrument = self.cache.instrument(self.config.instrument_id)
        self.account = self.portfolio.account(self.instrument.venue)
        self.trade_size = self.instrument.make_qty(self.config.trade_size)

        self.subscribe_trade_ticks(instrument_id=self.config.instrument_id)


    def on_stop(self):
        self.log.info("Stop of MACDStrategy")
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_trade_ticks(instrument_id=self.config.instrument_id)

    
    def on_order_accepted(self, event: OrderAccepted):
        if self.__stop_loss_order is not None:
            if self.__stop_loss_order.client_order_id == event.client_order_id:
                self.log.info(f"After stop loss order accepted balances locked: " + 
                              f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.MAGENTA)
                return

        if self.__take_profit_order is not None:
            if self.__take_profit_order.client_order_id == event.client_order_id:
                self.log.info(f"After take profit order accepted balances locked: " + 
                              f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.MAGENTA)
                return

        self.log.info(f"After unidentified order accepted balances locked: " + 
            f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.MAGENTA)
                
        
        

    def on_trade_tick(self, tick: TradeTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available
        self.macd.handle_trade_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry(tick)
        self.check_for_exit()


    def check_for_entry(self, tick: TradeTick):
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT
            
            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    
    def on_position_opened(self, event: PositionOpened):
        self.position = self.cache.position(event.position_id)

        if self.position.side == PositionSide.LONG:
            inverse_order_side = OrderSide.SELL

            take_profit_price = self.instrument.make_price(
                self.position_open_price * (1 + 0.05)
            )
            stop_loss_trigger_price = self.instrument.make_price(
                self.position_open_price * (1 - 0.05)
            )
        else:
            inverse_order_side = OrderSide.BUY

            take_profit_price = self.instrument.make_price(
                self.position_open_price * (1 - 0.05)
            )
            stop_loss_trigger_price = self.instrument.make_price(
                self.position_open_price * (1 + 0.05)
            )


        self.__stop_loss_order = self.order_factory.stop_market(
            self.config.instrument_id,
            inverse_order_side,
            event.quantity,
            trigger_price=stop_loss_trigger_price,
            trigger_type=TriggerType.LAST_PRICE,
            reduce_only=True,
            tags=["STOP"],
            time_in_force=TimeInForce.GTC,
            expire_time=None,
            quote_quantity=False,
            emulation_trigger=TriggerType.NO_TRIGGER,
            trigger_instrument_id=None,
            exec_algorithm_id=None,
            exec_algorithm_params=None
        )
        self.submit_order(self.__stop_loss_order)


        self.__take_profit_order = self.order_factory.limit(
            self.config.instrument_id,
            inverse_order_side,
            event.quantity,
            take_profit_price,
            reduce_only=True,
            post_only=False,
            tags=["TAKE_PROFIT"],
            time_in_force=TimeInForce.GTC,
            expire_time=None,
            quote_quantity=False,
            emulation_trigger=TriggerType.NO_TRIGGER,
            trigger_instrument_id=None,
            exec_algorithm_id=None,
            exec_algorithm_params=None,
            display_qty=None
        )
        self.submit_order(self.__take_profit_order)
    

    def on_dispose(self):
        pass  # Do nothing else

In [7]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
    chunk_size=1024*32
)

In [8]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

results: list[BacktestResult] = node.run()

2025-04-11T18:11:38.897055539Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-11T18:11:38.897080662Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-04-11T18:11:38.897084137Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-04-11T18:11:38.897086185Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2025. All rights reserved.
2025-04-11T18:11:38.897087558Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-11T18:11:38.897088902Z [INFO] BACKTESTER-001.BacktestEngine: 
2025-04-11T18:11:38.897090018Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-04-11T18:11:38.897092824Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-04-11T18:11:38.897094234Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀���
2025-04-11T18:

In [9]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

 account_id=BINANCE-001, opening_order_id=O-20241210-000008-001-000-6, closing_order_id=None, entry=BUY, side=LONG, signed_qty=500.0, quantity=500.0, peak_qty=500.0, currency=USDT, avg_px_open=11.92819982, avg_px_close=0.0, realized_return=0.00000, realized_pnl=-2.98204996 USDT, unrealized_pnl=0.00009000 USDT, ts_opened=1733788808218000000, ts_last=1733788808218000000, ts_closed=0, duration_ns=0)
2024-12-10T00:00:08.218000000Z [DEBUG] BACKTESTER-001.Cache: Indexed ClientOrderId('O-20241210-000008-001-000-7') with VenueOrderId('BINANCE-1-006')
2024-12-10T00:00:08.218000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=0.17375664 USDT
2024-12-10T00:00:08.218000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=94.80294540 USDT, locked=15.38221164 USDT, free=79.42073376 USDT)], margins=[MarginBalance(initial=0.17375664 USDT, maintenance=15.2084550

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,exec_spawn_id,tags,init_id,ts_init,ts_last,price,expire_time_ns,is_post_only,display_qty,trigger_instrument_id
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20241210-000005-001-000-1,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-001,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-002,MARKET,SELL,500.0,...,None,None,6d36a186-37b5-4b7a-98bc-ae3aedebd161,2024-12-10 00:00:05.048000+00:00,2024-12-10 00:00:05.048000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000005-001-000-3,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-003,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-047,LIMIT,BUY,1.0,...,None,[TAKE_PROFIT],fd33fa41-08cc-496d-8ee6-5abcb0fce06f,2024-12-10 00:00:05.048000+00:00,2024-12-10 03:25:29.327000+00:00,11.33890,NaN,False,NaN,NaN
O-20241210-000008-001-000-4,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-004,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-004,MARKET,BUY,500.0,...,None,None,1362e5bd-0e8f-4dde-95cc-150fe6f7ad60,2024-12-10 00:00:08.205000+00:00,2024-12-10 00:00:08.205000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000008-001-000-6,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-005,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-006,MARKET,BUY,500.0,...,None,None,2152a25e-7077-4000-88bc-705d5d401c77,2024-12-10 00:00:08.218000+00:00,2024-12-10 00:00:08.218000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000009-001-000-10,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-009,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-010,MARKET,SELL,500.0,...,None,None,064ebae6-e331-4d0a-b035-e77c1659da32,2024-12-10 00:00:09.609000+00:00,2024-12-10 00:00:09.609000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000009-001-000-9,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-008,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-008,MARKET,SELL,500.0,...,None,None,0c2e9389-f4b3-4481-a254-85532c1bb5ef,2024-12-10 00:00:09.609000+00:00,2024-12-10 00:00:09.609000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000010-001-000-13,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-012,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-012,MARKET,BUY,500.0,...,None,None,c3d9465b-7438-40a2-b709-da1a08928452,2024-12-10 00:00:10.101000+00:00,2024-12-10 00:00:10.101000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000010-001-000-15,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-013,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-014,MARKET,BUY,500.0,...,None,None,2bd1b566-b134-42a5-b92a-08e951830499,2024-12-10 00:00:10.101000+00:00,2024-12-10 00:00:10.101000+00:00,NaN,NaN,NaN,NaN,NaN
O-20241210-000010-001-000-18,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-016,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-016,MARKET,SELL,500.0,...,None,None,313f97ef-d47d-4aef-b527-8c92c769f221,2024-12-10 00:00:10.163000+00:00,2024-12-10 00:00:10.163000+00:00,NaN,NaN,NaN,NaN,NaN


BUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.92950,0.9,SELLER,245141139,1733788809609000000)
2024-12-10T00:00:09.609000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.92900,32.0,SELLER,245141140,1733788809609000000)
2024-12-10T00:00:09.908000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.92900,1.3,SELLER,245141141,1733788809908000000)
2024-12-10T00:00:10.043000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93030,0.9,BUYER,245141142,1733788810043000000)
2024-12-10T00:00:10.043000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93060,3.1,BUYER,245141143,1733788810043000000)
2024-12-10T00:00:10.099000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93060,33.3,SELLER,245

In [10]:
engine.trader.generate_positions_report()

000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.91210,30.4,SELLER,245141276,1733788810100000000)
2024-12-10T00:00:10.100000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.91200,1.4,SELLER,245141277,1733788810100000000)
2024-12-10T00:00:10.100000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.91180,39.6,SELLER,245141278,1733788810100000000)
2024-12-10T00:00:10.100000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.91170,1.3,SELLER,245141279,1733788810100000000)
2024-12-10T00:00:10.100000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.91160,16.8,SELLER,245141280,1733788810100000000)
2024-12-10T00:00:10.100000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.91150,

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
APTUSDT-PERP.BINANCE-MACDStrategy-000-71f6e74b-572e-4076-8486-8fe4a23f1013,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000005-001-000-1,O-20241210-000008-001-000-4,SELL,FLAT,0.0,500.0,1733788805048000000,2024-12-10 00:00:05.048000+00:00,1733788808205000000,2024-12-10 00:00:08.204999936+00:00,3.157000e+09,11.935600,11.928098,[5.96592464 USDT],0.00063,-2.21500464 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-e0d8da33-14e0-477f-b4d1-0842aef8caf3,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000008-001-000-6,O-20241210-000009-001-000-9,BUY,FLAT,0.0,500.0,1733788808218000000,2024-12-10 00:00:08.218000+00:00,1733788809609000000,2024-12-10 00:00:09.608999936+00:00,1.391000e+09,11.928200,11.929900,[5.96452503 USDT],0.00014,-5.11429503 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-87fbceac-7efd-4fe7-8ab8-ebbc8caa9ef6,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000009-001-000-10,O-20241210-000010-001-000-13,SELL,FLAT,0.0,500.0,1733788809609000000,2024-12-10 00:00:09.609000+00:00,1733788810101000000,2024-12-10 00:00:10.100999936+00:00,4.920000e+08,11.929800,11.930693,[5.96512339 USDT],-0.00007,-6.41170339 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-d9e05c80-c982-4fc4-9ac5-43817d0fca0d,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-15,O-20241210-000010-001-000-18,BUY,FLAT,0.0,500.0,1733788810101000000,2024-12-10 00:00:10.101000+00:00,1733788810163000000,2024-12-10 00:00:10.163000064+00:00,6.200000e+07,11.928294,11.920617,[5.96222758 USDT],-0.00064,-9.80078758 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-5b3b56f7-4288-493b-9c33-d2fd8d6c9a2e,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-19,O-20241210-000010-001-000-22,SELL,FLAT,0.0,500.0,1733788810181000000,2024-12-10 00:00:10.181000+00:00,1733788810201000000,2024-12-10 00:00:10.200999936+00:00,2.000000e+07,11.919203,11.921592,[5.96019875 USDT],-0.00020,-7.15432875 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-ec5226f2-42f3-458a-8835-7cd4a8f25e09,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-23,O-20241210-000010-001-000-26,BUY,FLAT,0.0,500.0,1733788810242000000,2024-12-10 00:00:10.242000+00:00,1733788810265000000,2024-12-10 00:00:10.264999936+00:00,2.300000e+07,11.921697,11.919906,[5.96040062 USDT],-0.00015,-6.85593062 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-4e04cc94-2f19-4ee2-bac1-4e26d8329054,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-27,O-20241210-000010-001-000-30,SELL,FLAT,0.0,500.0,1733788810271000000,2024-12-10 00:00:10.271000+00:00,1733788810613000000,2024-12-10 00:00:10.612999936+00:00,3.420000e+08,11.920117,11.921496,[5.96040323 USDT],-0.00012,-6.64979323 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-417497dd-68c5-4441-8cb6-54e08dfe3c36,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-31,O-20241210-000011-001-000-34,BUY,FLAT,0.0,500.0,1733788810613000000,2024-12-10 00:00:10.613000+00:00,1733788811341000000,2024-12-10 00:00:11.340999936+00:00,7.280000e+08,11.921600,11.920700,[5.96057501 USDT],-0.00008,-6.41039501 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-c2565a4a-fff9-494c-8b22-e8f1b9a928f3,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000011-001-000-35,O-20241210-000011-001-000-38,SELL,FLAT,0.0,500.0,1733788811341000000,2024-12-10 00:00:11.341000+00:00,1733788811487000000,2024-12-10 00:00:11.487000064+00:00,1.460000e+08,11.920600,11.919699,[5.96007485 USDT],0.00008,-5.50948485 USDT


001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000010-001-000-16, account_id=BINANCE-001, ts_event=1733788810101000000)
2024-12-10T00:00:10.101000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000010-001-000-17, side=SELL, type=LIMIT, quantity=31.4, time_in_force=GTC, post_only=False, reduce_only=True, quote_quantity=False, options={'price': '12.52460', 'display_qty': None, 'expire_time_ns': 0}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=['TAKE_PROFIT'])
2024-12-10T00:00:10.101000000Z [DEBUG] BACKTESTER-001.Cache: Added LimitOrder(SELL 31.4 APTUSDT-PERP.BINANCE LIMIT @ 12.52460 GTC, status=INITIALIZED, client_order_id=O-20241210-000010-001-000-17, venue_order_i

In [11]:
engine.trader.generate_account_report(Venue("BINANCE"))

T, signed_qty=-500.0, quantity=500.0, peak_qty=500.0, currency=USDT, avg_px_open=11.91920336, avg_px_close=0.0, realized_return=0.00000, realized_pnl=-2.97980084 USDT, unrealized_pnl=0.00168000 USDT, ts_opened=1733788810181000000, ts_last=1733788810181000000, ts_closed=0, duration_ns=0)
2024-12-10T00:00:10.181000000Z [DEBUG] BACKTESTER-001.Cache: Indexed ClientOrderId('O-20241210-000010-001-000-20') with VenueOrderId('BINANCE-1-018')
2024-12-10T00:00:10.181000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=5.23343538 USDT
2024-12-10T00:00:10.181000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=73.47840852 USDT, locked=20.43041538 USDT, free=53.04799314 USDT)], margins=[MarginBalance(initial=5.23343538 USDT, maintenance=15.19698000 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=b04880da-e41b-4919-80b7-e7424b0e554a)
2024-12-10T00:00:1

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2024-12-10 00:00:04.998000+00:00,100.00000000,0E-8,100.00000000,USDT,BINANCE-001,MARGIN,USDT,[],True,{}
2024-12-10 00:00:05.048000+00:00,99.99403215,0E-8,99.99403215,USDT,BINANCE-001,MARGIN,USDT,[],False,{}
2024-12-10 00:00:05.048000+00:00,97.01609995,0.03043604,96.98566391,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '19.8971...",False,{}
2024-12-10 00:00:05.048000+00:00,97.01609995,15.28180575,81.73429420,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '19.8971...",False,{}
2024-12-10 00:00:05.048000+00:00,97.01609995,15.33963414,81.67646581,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '19.8971...",False,{}
...,...,...,...,...,...,...,...,...,...,...
2024-12-10 00:00:11.884000+00:00,40.89897648,35.04000136,5.85897512,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '19.8971...",False,{}
2024-12-10 00:00:12.479000+00:00,40.89172430,35.04000136,5.85172294,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '19.8971...",False,{}
2024-12-10 00:00:12.479000+00:00,36.82005652,35.01265139,1.80740513,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '19.8971...",False,{}
2024-12-10 00:00:12.482000+00:00,36.81469270,19.84557136,16.96912134,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '19.8971...",False,{}


:10.242000000Z [DEBUG] BACKTESTER-001.Cache: Indexed ClientOrderId('O-20241210-000010-001-000-23') with VenueOrderId('BINANCE-1-021')
2024-12-10T00:00:10.242000000Z [DEBUG] BACKTESTER-001.Portfolio: Calculated PnLs: []
2024-12-10T00:00:10.242000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=6.20361572 USDT
2024-12-10T00:00:10.242000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=69.20731565 USDT, locked=6.20361572 USDT, free=63.00369993 USDT)], margins=[MarginBalance(initial=6.20361572 USDT, maintenance=0.00000000 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=60b97d8c-2cef-40bc-a084-7b5a4c309f59)
2024-12-10T00:00:10.242000000Z [DEBUG] BACKTESTER-001.Portfolio: Updated OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000010-001-000-23, venue_order_id=BINANCE-1-021, account_id=BINANCE-001, trade_id=BINANCE-1